**Общая информация**

**Срок сдачи:** 13 марта 2017, 06:00 <br\>
**Штраф за опоздание:** -2 балла после 06:00 13 марта, -4 балла после 06:00 20 марта, -6 баллов после 06:00 27 марта

При отправлении ДЗ указывайте фамилию в названии файла
Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush

Необходимо в slack создать таск в приватный чат:
/todo Фамилия Имя ссылка на гитхаб @alkhamush
Пример:
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw1.ipynb @alkhamush

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (2 баллов)
Реализовать KNN в классе MyKNeighborsClassifier (обязательное условие: точность не ниже sklearn реализации)
Разберитесь самостоятельно, какая мера расстояния используется в KNeighborsClassifier дефолтно и реализуйте свой алгоритм именно с этой мерой. Самостоятельно разберитесь, как считается score из KNeighborsClassifier и реализуйте аналог в своём классе.

###### Задание 2 (2 балла)
Добиться скорости работы на fit,  predict и predict_proba сравнимой со sklearn 4 балла для iris и mnist
Для этого используем numpy

###### Задание 3 (2 балла)
Для iris найдите такой параметр n_neighbors, при котором выдаётся наилучший score. Нарисуйте график зависимости score от n_neighbors

###### Задание 3 (2 балла)
Выполнить требования pep8

###### Задание 5 (2 балла)
Описать для чего нужны следующие библиотеки/классы/функции (список будет ниже)


In [2]:
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn import datasets
from sklearn.base import ClassifierMixin
from sklearn.datasets import fetch_mldata
from sklearn.neighbors.base import NeighborsBase, KNeighborsMixin, SupervisedIntegerMixin 
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier



In [3]:
from collections import Counter
from scipy.spatial.distance import cdist
from scipy.stats import mode
import operator

class MyKNeighborsClassifier(NeighborsBase, KNeighborsMixin, SupervisedIntegerMixin, ClassifierMixin):
    
    def __init__(self, n_neighbors):
        self.k = n_neighbors
        self.X = np.zeros(0)
        self.y = np.zeros(0)
        self.N_classes = 0
    
    def fit(self, X, y):
        self.X = X
        self.y = y.astype(int)
        self.N_classes = len(Counter(y.tolist()))
        
        
    def predict(self, X_test):
        dst = cdist(X_test, self.X)
        y_metrics = self.y[np.argsort(dst)[:, :self.k]]
        return mode(y_metrics, axis = 1).mode.reshape((-1,))
        
            
                
                
            
    
    def predict_proba(self, X_test):
        dst = cdist(X_test, self.X)
        ans = np.zeros((X_test.shape[0], self.N_classes))
        y_metrics = self.y[np.argsort(dst)[:, :self.k]]
        for i in range(y_metrics.shape[0]):
            for j in range(y_metrics.shape[1]):
                ans[i][y_metrics[i][j].astype(int)] += 1 
        ans /= self.k
        return ans
    
    def score(self, X, y):
        print self.predict(X),  y
        return 1 - (0. + np.sum(np.not_equal(self.predict(X), y))) / y.shape[0]
        

**IRIS**

In [4]:
iris = datasets.load_iris()

In [5]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)


In [6]:
clf = KNeighborsClassifier(n_neighbors=10) 
my_clf = MyKNeighborsClassifier(n_neighbors=10)
%time clf.fit(X_train, y_train)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.44 ms


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights='uniform')

In [7]:
%time my_clf.fit(X_train, y_train)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 279 µs


In [8]:
%time clf.predict(X_test)

CPU times: user 0 ns, sys: 4 ms, total: 4 ms
Wall time: 3.11 ms


array([2, 1, 1, 0, 0, 1, 2, 2, 0, 2, 2, 0, 0, 2, 1])

In [9]:
%time my_clf.predict(X_test)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 1.71 ms


array([2, 1, 1, 0, 0, 1, 2, 2, 0, 2, 2, 0, 0, 2, 1])

In [10]:
%time clf.predict_proba(X_test)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 3.83 ms


array([[ 0. ,  0. ,  1. ],
       [ 0. ,  1. ,  0. ],
       [ 0. ,  1. ,  0. ],
       [ 1. ,  0. ,  0. ],
       [ 1. ,  0. ,  0. ],
       [ 0. ,  0.9,  0.1],
       [ 0. ,  0.2,  0.8],
       [ 0. ,  0. ,  1. ],
       [ 1. ,  0. ,  0. ],
       [ 0. ,  0.4,  0.6],
       [ 0. ,  0. ,  1. ],
       [ 1. ,  0. ,  0. ],
       [ 1. ,  0. ,  0. ],
       [ 0. ,  0. ,  1. ],
       [ 0. ,  1. ,  0. ]])

In [11]:
%time my_clf.predict_proba(X_test)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 4.82 ms


array([[ 0. ,  0. ,  1. ],
       [ 0. ,  1. ,  0. ],
       [ 0. ,  1. ,  0. ],
       [ 1. ,  0. ,  0. ],
       [ 1. ,  0. ,  0. ],
       [ 0. ,  0.9,  0.1],
       [ 0. ,  0.2,  0.8],
       [ 0. ,  0. ,  1. ],
       [ 1. ,  0. ,  0. ],
       [ 0. ,  0.4,  0.6],
       [ 0. ,  0. ,  1. ],
       [ 1. ,  0. ,  0. ],
       [ 1. ,  0. ,  0. ],
       [ 0. ,  0. ,  1. ],
       [ 0. ,  1. ,  0. ]])

In [12]:
clf.score(X_test, y_test)

0.93333333333333335

In [13]:
my_clf.score(X_test, y_test)

[2 1 1 0 0 1 2 2 0 2 2 0 0 2 1] [2 1 1 0 0 1 1 2 0 2 2 0 0 2 1]


0.9333333333333333

In [14]:
res = [0]
for i in range(2, 15):
    CL = MyKNeighborsClassifier(n_neighbors=i)
    CL.fit(X_train, y_train)
    print CL.score(X_test, y_test)

[2 1 1 0 0 1 2 2 0 2 2 0 0 2 1] [2 1 1 0 0 1 1 2 0 2 2 0 0 2 1]
0.933333333333
[2 1 1 0 0 1 2 2 0 2 2 0 0 2 1] [2 1 1 0 0 1 1 2 0 2 2 0 0 2 1]
0.933333333333
[2 1 1 0 0 1 2 2 0 2 2 0 0 2 1] [2 1 1 0 0 1 1 2 0 2 2 0 0 2 1]
0.933333333333
[2 1 1 0 0 1 2 2 0 2 2 0 0 2 1] [2 1 1 0 0 1 1 2 0 2 2 0 0 2 1]
0.933333333333
[2 1 1 0 0 1 2 2 0 2 2 0 0 2 1] [2 1 1 0 0 1 1 2 0 2 2 0 0 2 1]
0.933333333333
[2 1 1 0 0 1 2 2 0 2 2 0 0 2 1] [2 1 1 0 0 1 1 2 0 2 2 0 0 2 1]
0.933333333333
[2 1 1 0 0 1 2 2 0 2 2 0 0 2 1] [2 1 1 0 0 1 1 2 0 2 2 0 0 2 1]
0.933333333333
[2 1 1 0 0 1 2 2 0 2 2 0 0 2 1] [2 1 1 0 0 1 1 2 0 2 2 0 0 2 1]
0.933333333333
[2 1 1 0 0 1 2 2 0 2 2 0 0 2 1] [2 1 1 0 0 1 1 2 0 2 2 0 0 2 1]
0.933333333333
[2 1 1 0 0 1 2 2 0 2 2 0 0 2 1] [2 1 1 0 0 1 1 2 0 2 2 0 0 2 1]
0.933333333333
[2 1 1 0 0 1 2 2 0 1 2 0 0 2 1] [2 1 1 0 0 1 1 2 0 2 2 0 0 2 1]
0.866666666667
[2 1 1 0 0 1 2 2 0 2 2 0 0 2 1] [2 1 1 0 0 1 1 2 0 2 2 0 0 2 1]
0.933333333333
[2 1 1 0 0 1 2 2 0 2 2 0 0 2 1] [2 1 1 0 0 1 1 2 0 2

In [13]:
# Задание 3

**MNIST**

In [14]:
mnist = fetch_mldata('MNIST original')

IOError: could not read bytes

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(mnist.data, mnist.target, test_size=0.1, stratify=mnist.target)

In [ ]:
clf = KNeighborsClassifier(n_neighbors=2)
my_clf = MyKNeighborsClassifier(n_neighbors=2)

In [15]:
%time clf.fit(X_train, y_train)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 1.73 ms


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights='uniform')

In [16]:
%time my_clf.fit(X_train, y_train)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 252 µs


In [39]:
%time clf.predict(X_test)

CPU times: user 7min 41s, sys: 16 ms, total: 7min 41s
Wall time: 7min 41s


array([5., 0., 8., ..., 2., 2., 7.])

In [40]:
%time my_clf.predict(X_test)

CPU times: user 3min 13s, sys: 1.49 s, total: 3min 15s
Wall time: 3min 15s


array([5., 0., 8., ..., 2., 2., 7.])

In [41]:
%time clf.predict_proba(X_test)

CPU times: user 7min 51s, sys: 12 ms, total: 7min 51s
Wall time: 7min 51s


array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [42]:
%time my_clf.predict_proba(X_test)

CPU times: user 3min 14s, sys: 1.71 s, total: 3min 16s
Wall time: 3min 16s


array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [43]:
clf.score(X_test, y_test)

0.9691428571428572

In [44]:
my_clf.score(X_test, y_test)

[5. 0. 8. ... 2. 2. 7.] [5. 0. 8. ... 2. 2. 7.]


0.9998571428571429

Задание 5

In [ ]:
# seaborn - визуализатор python-данных, настройка над matplotlib
# matplotlib
# train_test_split
# Pipelin%load_ext e (from sklearn.pipeline import Pipeline)
# StandardScaler (from sklearn.preprocessing import StandardScaler)
# ClassifierMixin
# NeighborsBase
# KNeighborsMixin
# SupervisedIntegerMixin